<a href="https://colab.research.google.com/github/marco-siino/gpt_networking/blob/main/Q1/google_codegemma_7b_it_LogKnowledgeExtraction_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries and load the model.

In [4]:
import os
import random
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

model_id = "cache/models--google--codegemma-7b-it/snapshots/9b1d9231388358c04d90bd003458f5070d97db44/"

#model_id = "google/gemma-7b-it"
dtype = torch.bfloat16

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=dtype,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
!nvidia-smi

Wed Jun 26 11:10:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM-64GB            On | 00000000:1D:00.0 Off |                    0 |
| N/A   44C    P0               74W / 457W|   5213MiB / 65536MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

# Following code to kill the processes using the GPUs.

In [2]:
!kill -9 154312

# Gemma 7B Instruct example usage on the local model.

In [6]:
log_sample_1 = """Given the following LOG, how many frames are received per each dev_eui?

LOG:
time: 2019-06-08 06:16:06.646000, freq: 868.1, chan: 21, lsnr: -2.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4580
time: 2019-06-08 06:19:55.146000, freq: 868.3, chan: 22, lsnr: -6.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 4323
time: 2019-06-08 06:33:52.597000, freq: 868.1, chan: 21, lsnr: 5.0, rssi: -107.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4574
time: 2019-06-08 06:35:17.832000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 4395
time: 2019-06-08 06:40:37.829000, freq: 868.3, chan: 22, lsnr: 15.0, rssi: -83.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 23, FCnt: 4525
time: 2019-06-08 06:42:23.968000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -88.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 623
time: 2019-06-08 07:08:17.919000, freq: 868.5, chan: 23, lsnr: 14.0, rssi: -82.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 125, size: 23, FCnt: 74
time: 2019-06-08 07:09:17.204000, freq: 868.3, chan: 22, lsnr: -2.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 25, FCnt: 4325
time: 2019-06-08 07:10:35.004000, freq: 868.1, chan: 21, lsnr: 9.0, rssi: -106.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4568
time: 2019-06-08 07:16:06.292000, freq: 868.5, chan: 23, lsnr: 1.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4581
time: 2019-06-08 07:19:48.095000, freq: 868.3, chan: 22, lsnr: -8.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 4324
time: 2019-06-08 07:19:54.233000, freq: 868.5, chan: 23, lsnr: -4.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 4324
time: 2019-06-08 07:33:51.891000, freq: 868.3, chan: 22, lsnr: 9.0, rssi: -105.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4575
time: 2019-06-08 07:35:17.507000, freq: 868.5, chan: 23, lsnr: 1.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 4396
time: 2019-06-08 07:40:37.002000, freq: 868.3, chan: 22, lsnr: 15.0, rssi: -85.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 23, FCnt: 4526
time: 2019-06-08 07:42:23.134000, freq: 868.1, chan: 21, lsnr: 15.0, rssi: -86.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 624
time: 2019-06-08 08:08:17.160000, freq: 868.1, chan: 21, lsnr: 8.0, rssi: -82.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 125, size: 23, FCnt: 75
time: 2019-06-08 08:09:16.921000, freq: 868.1, chan: 21, lsnr: 0.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 25, FCnt: 4326
time: 2019-06-08 08:09:23.048000, freq: 868.5, chan: 23, lsnr: -3.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 25, FCnt: 4326
time: 2019-06-08 08:09:29.242000, freq: 868.1, chan: 21, lsnr: 0.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 25, FCnt: 4326
time: 2019-06-08 08:10:34.179000, freq: 868.5, chan: 23, lsnr: -6.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4569

"""
answer_1 = """
For DEVICE dev_eui:120 all received frames are 2.
For DEVICE dev_eui:121 all received frames are 2.
For DEVICE dev_eui:122 all received frames are 2.
For DEVICE dev_eui:123 all received frames are 2.
For DEVICE dev_eui:124 all received frames are 3.
For DEVICE dev_eui:125 all received frames are 2.
For DEVICE dev_eui:126 all received frames are 2.
For DEVICE dev_eui:127 all received frames are 2.
For DEVICE dev_eui:128 all received frames are 4.
"""

log_sample_2 = """Given the following LOG, how many frames are received per each dev_eui?

LOG:
time: 2019-06-16 06:14:46.400000, freq: 868.1, chan: 21, lsnr: -8.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4780
time: 2019-06-16 06:17:03.760000, freq: 868.3, chan: 22, lsnr: -1.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 25, FCnt: 4523
time: 2019-06-16 06:20:19.487000, freq: 868.1, chan: 21, lsnr: 0.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 129, size: 23, FCnt: 71
time: 2019-06-16 06:20:25.627000, freq: 868.3, chan: 22, lsnr: 2.0, rssi: -111.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 129, size: 23, FCnt: 71
time: 2019-06-16 06:31:07.333000, freq: 868.3, chan: 22, lsnr: 5.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4774
time: 2019-06-16 06:34:02.275000, freq: 868.3, chan: 22, lsnr: -8.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 4595
time: 2019-06-16 06:37:52.472000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -84.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 25, FCnt: 4725
time: 2019-06-16 06:40:00.992000, freq: 868.1, chan: 21, lsnr: 15.0, rssi: -86.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 823
time: 2019-06-16 07:05:22.808000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -82.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 125, size: 25, FCnt: 274
time: 2019-06-16 07:07:46.100000, freq: 868.1, chan: 21, lsnr: 4.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4768
time: 2019-06-16 07:08:02.078000, freq: 868.1, chan: 21, lsnr: -4.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 25, FCnt: 4525
time: 2019-06-16 07:08:08.281000, freq: 868.5, chan: 23, lsnr: -4.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 25, FCnt: 4525
time: 2019-06-16 07:08:14.413000, freq: 868.5, chan: 23, lsnr: -5.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 25, FCnt: 4525
time: 2019-06-16 07:17:02.972000, freq: 868.3, chan: 22, lsnr: -4.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 25, FCnt: 4524
time: 2019-06-16 07:17:09.104000, freq: 868.1, chan: 21, lsnr: 0.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 25, FCnt: 4524
time: 2019-06-16 07:17:15.300000, freq: 868.3, chan: 22, lsnr: -4.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 25, FCnt: 4524
time: 2019-06-16 07:20:24.961000, freq: 868.5, chan: 23, lsnr: -8.0, rssi: -108.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 129, size: 23, FCnt: 72
time: 2019-06-16 07:31:06.447000, freq: 868.3, chan: 22, lsnr: -2.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4775
time: 2019-06-16 07:37:51.557000, freq: 868.1, chan: 21, lsnr: 15.0, rssi: -82.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 23, FCnt: 4726
time: 2019-06-16 07:40:00.203000, freq: 868.1, chan: 21, lsnr: 15.0, rssi: -91.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 824
time: 2019-06-16 08:05:21.842000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -81.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 125, size: 25, FCnt: 275
time: 2019-06-16 08:07:58.174000, freq: 868.1, chan: 21, lsnr: -7.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 4526

"""
answer_2 = """
For DEVICE dev_eui:120 all received frames are 2.
For DEVICE dev_eui:121 all received frames are 2.
For DEVICE dev_eui:122 all received frames are 1.
For DEVICE dev_eui:123 all received frames are 1.
For DEVICE dev_eui:124 all received frames are 4.
For DEVICE dev_eui:125 all received frames are 2.
For DEVICE dev_eui:126 all received frames are 1.
For DEVICE dev_eui:127 all received frames are 2.
For DEVICE dev_eui:128 all received frames are 4.
For DEVICE dev_eui:129 all received frames are 3.
"""

log_sample_3 = """Given the following LOG, how many frames are received per each dev_eui?

LOG:
time: 2019-06-19 22:15:49.016000, freq: 868.3, chan: 22, lsnr: -4.0, rssi: -110.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 25, FCnt: 4615
time: 2019-06-19 22:19:26.217000, freq: 868.3, chan: 22, lsnr: 12.0, rssi: -103.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 129, size: 23, FCnt: 163
time: 2019-06-19 22:19:32.410000, freq: 868.3, chan: 22, lsnr: 10.0, rssi: -103.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 129, size: 23, FCnt: 163
time: 2019-06-19 22:29:51.788000, freq: 868.3, chan: 22, lsnr: 8.0, rssi: -105.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4866
time: 2019-06-19 22:33:27.661000, freq: 868.3, chan: 22, lsnr: -3.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 4687
time: 2019-06-19 22:36:39.477000, freq: 868.3, chan: 22, lsnr: 15.0, rssi: -88.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 23, FCnt: 4817
time: 2019-06-19 22:36:45.605000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -87.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 23, FCnt: 4817
time: 2019-06-19 22:38:53.458000, freq: 868.3, chan: 22, lsnr: 15.0, rssi: -93.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 915
time: 2019-06-19 23:04:02.955000, freq: 868.1, chan: 21, lsnr: 15.0, rssi: -79.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 125, size: 23, FCnt: 366
time: 2019-06-19 23:06:28.693000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -110.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4860
time: 2019-06-19 23:07:21.668000, freq: 868.3, chan: 22, lsnr: -3.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 25, FCnt: 4617
time: 2019-06-19 23:14:14.339000, freq: 868.5, chan: 23, lsnr: -5.0, rssi: -111.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 25, FCnt: 4872
time: 2019-06-19 23:15:48.128000, freq: 868.3, chan: 22, lsnr: -2.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 25, FCnt: 4616
time: 2019-06-19 23:15:54.332000, freq: 868.3, chan: 22, lsnr: -3.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 25, FCnt: 4616
time: 2019-06-19 23:16:00.529000, freq: 868.5, chan: 23, lsnr: -7.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 25, FCnt: 4616
time: 2019-06-19 23:19:25.520000, freq: 868.5, chan: 23, lsnr: 10.0, rssi: -103.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 129, size: 25, FCnt: 164
time: 2019-06-19 23:29:50.922000, freq: 868.5, chan: 23, lsnr: 8.0, rssi: -106.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4867
time: 2019-06-19 23:33:27.343000, freq: 868.5, chan: 23, lsnr: -1.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 4688
time: 2019-06-19 23:36:38.619000, freq: 868.1, chan: 21, lsnr: 15.0, rssi: -88.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 23, FCnt: 4818
time: 2019-06-19 23:36:44.813000, freq: 868.1, chan: 21, lsnr: 14.0, rssi: -87.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 23, FCnt: 4818
time: 2019-06-19 23:38:52.544000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -93.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 916
time: 2019-06-20 00:04:01.992000, freq: 868.1, chan: 21, lsnr: 15.0, rssi: -79.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 125, size: 23, FCnt: 367
time: 2019-06-20 00:06:27.838000, freq: 868.3, chan: 22, lsnr: 9.0, rssi: -105.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4861
time: 2019-06-20 00:07:21.274000, freq: 868.5, chan: 23, lsnr: -7.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 4618
time: 2019-06-20 00:07:27.470000, freq: 868.3, chan: 22, lsnr: -4.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 4618
time: 2019-06-20 00:07:33.606000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 4618

"""
answer_3 = """
For DEVICE dev_eui:120 all received frames are 2.
For DEVICE dev_eui:121 all received frames are 2.
For DEVICE dev_eui:122 all received frames are 1.
For DEVICE dev_eui:123 all received frames are 2.
For DEVICE dev_eui:124 all received frames are 4.
For DEVICE dev_eui:125 all received frames are 2.
For DEVICE dev_eui:126 all received frames are 2.
For DEVICE dev_eui:127 all received frames are 4.
For DEVICE dev_eui:128 all received frames are 4.
For DEVICE dev_eui:129 all received frames are 3.
"""

log_test = """

Given the following LOG, how many frames are received per each dev_eui?

LOG:
time: 2019-06-10 08:15:47.577000, freq: 868.3, chan: 22, lsnr: -8.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4632
time: 2019-06-10 08:33:09.794000, freq: 868.3, chan: 22, lsnr: 9.0, rssi: -104.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4626
time: 2019-06-10 08:33:15.987000, freq: 868.1, chan: 21, lsnr: 12.0, rssi: -103.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4626
time: 2019-06-10 08:35:00.110000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 4447
time: 2019-06-10 08:39:47.076000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -88.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 26, FCnt: 4577
time: 2019-06-10 08:40:15.677000, freq: 868.5, chan: 23, lsnr: 14.0, rssi: -86.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 25, FCnt: 4578
time: 2019-06-10 08:41:45.318000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -90.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 675
time: 2019-06-10 09:07:32.593000, freq: 868.1, chan: 21, lsnr: 15.0, rssi: -82.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 125, size: 25, FCnt: 126
time: 2019-06-10 09:08:46.623000, freq: 868.3, chan: 22, lsnr: -4.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 24, FCnt: 4377
time: 2019-06-10 09:08:52.817000, freq: 868.1, chan: 21, lsnr: -6.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 24, FCnt: 4377
time: 2019-06-10 09:08:59.012000, freq: 868.1, chan: 21, lsnr: -4.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 24, FCnt: 4377
time: 2019-06-10 09:09:27.817000, freq: 868.5, chan: 23, lsnr: -3.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 4378
time: 2019-06-10 09:09:33.946000, freq: 868.1, chan: 21, lsnr: -5.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 4378
time: 2019-06-10 09:09:40.143000, freq: 868.3, chan: 22, lsnr: -4.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 4378
time: 2019-06-10 09:09:51.186000, freq: 868.1, chan: 21, lsnr: 6.0, rssi: -108.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4620
time: 2019-06-10 09:15:47.450000, freq: 868.3, chan: 22, lsnr: -8.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4633
time: 2019-06-10 09:19:04.570000, freq: 868.3, chan: 22, lsnr: -4.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 4376
time: 2019-06-10 09:19:16.956000, freq: 868.5, chan: 23, lsnr: -7.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 4376
time: 2019-06-10 09:33:00.703000, freq: 868.1, chan: 21, lsnr: 5.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 24, FCnt: 4627
time: 2019-06-10 09:33:06.902000, freq: 868.3, chan: 22, lsnr: 7.0, rssi: -107.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 24, FCnt: 4627
time: 2019-06-10 09:33:33.088000, freq: 868.3, chan: 22, lsnr: 11.0, rssi: -103.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4628
time: 2019-06-10 09:33:39.286000, freq: 868.3, chan: 22, lsnr: 2.0, rssi: -111.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4628
time: 2019-06-10 09:34:59.767000, freq: 868.1, chan: 21, lsnr: 0.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 25, FCnt: 4448
time: 2019-06-10 09:39:54.277000, freq: 868.1, chan: 21, lsnr: 15.0, rssi: -84.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 23, FCnt: 4579
time: 2019-06-10 09:41:44.594000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -90.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 676
time: 2019-06-10 10:07:31.414000, freq: 868.1, chan: 21, lsnr: 13.0, rssi: -81.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 125, size: 26, FCnt: 127
time: 2019-06-10 10:07:55.435000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -81.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 125, size: 23, FCnt: 128
time: 2019-06-10 10:08:54.046000, freq: 868.5, chan: 23, lsnr: -5.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 4379
time: 2019-06-10 10:09:00.175000, freq: 868.5, chan: 23, lsnr: -1.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 4379
time: 2019-06-10 10:09:06.375000, freq: 868.5, chan: 23, lsnr: -2.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 4379
time: 2019-06-10 10:09:50.252000, freq: 868.1, chan: 21, lsnr: -1.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4621

"""


#chat = [
#    { "role": "user", "content": "Write a hello world program" },
#]

chat = [
    { "role": "user", "content": log_sample_1 },
    { "role": "model", "content": answer_1 },
    #{ "role": "user", "content": log_sample_2 },
    #{ "role": "model", "content": answer_2 },
    #{ "role": "user", "content": log_sample_3 },
    #{ "role": "model", "content": answer_3 },
    { "role": "user", "content": log_test },
]


prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

print(prompt)

inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=1500)
print(tokenizer.decode(outputs[0]))

<bos><start_of_turn>user
Given the following LOG, how many frames are received per each dev_eui?

LOG:
time: 2019-06-08 06:16:06.646000, freq: 868.1, chan: 21, lsnr: -2.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4580
time: 2019-06-08 06:19:55.146000, freq: 868.3, chan: 22, lsnr: -6.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 4323
time: 2019-06-08 06:33:52.597000, freq: 868.1, chan: 21, lsnr: 5.0, rssi: -107.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4574
time: 2019-06-08 06:35:17.832000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 4395
time: 2019-06-08 06:40:37.829000, freq: 868.3, chan: 22, lsnr: 15.0, rssi: -83.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 23, FCnt: 4525
time: 2019-06-08 06:42:23.968000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -88.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 623
time: 2019-06-08 07:

# Generic Gemma 7B usage.

In [ ]:
# pip install accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_path = "/leonardo/home/userexternal/msiino00/.cache/huggingface/hub/models--google--gemma-7b-it/snapshots/18329f019fb74ca4b24f97371785268543d687d2"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype=torch.float16,
    revision="float16",
)

input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, )
print(tokenizer.decode(outputs[0]))


`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/leonardo/home/userexternal/msiino00/clean-updated-env/lib/python3.10/site-packages/transformers/generation/utils.py:1244: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<bos>Write me a poem about Machine Learning.

In the realm of data, a tale unfolds,


# Organize the dataset.

In [ ]:
!unzip prompt_Q3-2024-06-13.zip

Archive:  prompt_Q3-2024-06-13.zip
  inflating: prompt_Q3/prompt_n_hours=2_skip_hours=1076.txt  
  inflating: prompt_Q3/prompt_n_hours=2_skip_hours=1074.txt  
  inflating: prompt_Q3/prompt_n_hours=2_skip_hours=1072.txt  
  inflating: prompt_Q3/prompt_n_hours=2_skip_hours=1088.txt  
  inflating: prompt_Q3/prompt_n_hours=2_skip_hours=1090.txt  
  inflating: prompt_Q3/prompt_n_hours=2_skip_hours=1082.txt  
  inflating: prompt_Q3/prompt_n_hours=2_skip_hours=1070.txt  
  inflating: prompt_Q3/prompt_n_hours=2_skip_hours=1078.txt  
  inflating: prompt_Q3/prompt_n_hours=2_skip_hours=1086.txt  
  inflating: prompt_Q3/prompt_n_hours=2_skip_hours=1084.txt  
  inflating: prompt_Q3/prompt_n_hours=2_skip_hours=1080.txt  
  inflating: prompt_Q3/prompt_n_hours=2_skip_hours=1036.txt  
  inflating: prompt_Q3/prompt_n_hours=2_skip_hours=1044.txt  
  inflating: prompt_Q3/prompt_n_hours=2_skip_hours=998.txt  
  inflating: prompt_Q3/prompt_n_hours=2_skip_hours=1030.txt  
  inflating: prompt_Q3/prompt_n_hour

In [7]:
folder_path="log_knowledge_extraction/prompt_Q1-2024-05-20"
# List all files in the folder
files = os.listdir(folder_path)

# Sort the files alphabetically
files.sort()

# Function to pre-process the few shots.

In [8]:
# The following function add the Mistral Tags within the file content.
def preprocess_sample(file_contents):

                log_sample ="Given the following LOG, how many frames are received per each dev_eui?\n\nLOG:\n"

                parts = file_contents.split('QUESTION:')
                log_sample += parts[0]
                log_sample += "ANSWER:"

                parts = file_contents.split('ANSWER:')
                answer = parts[1]

                question = { "role": "user", "content": log_sample }
                answer = { "role": "model", "content": answer }

                return question,answer


In [9]:
file_contents = """
time: 2019-05-23 12:16:10.162000, freq: 868.1, chan: 21, lsnr: 8.0, rssi: -105.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4173
time: 2019-05-23 12:18:43.187000, freq: 868.5, chan: 23, lsnr: -6.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4186
time: 2019-05-23 12:25:19.676000, freq: 868.5, chan: 23, lsnr: -2.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3930
time: 2019-05-23 12:25:32.075000, freq: 868.3, chan: 22, lsnr: -7.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3930
time: 2019-05-23 12:37:55.014000, freq: 868.5, chan: 23, lsnr: 4.0, rssi: -110.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 4001
time: 2019-05-23 12:39:21.253000, freq: 868.1, chan: 21, lsnr: 0.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4181
time: 2019-05-23 12:46:05.929000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -87.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 23, FCnt: 4132
time: 2019-05-23 12:47:53.825000, freq: 868.1, chan: 21, lsnr: 15.0, rssi: -89.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 230
time: 2019-05-23 13:11:48.913000, freq: 868.5, chan: 23, lsnr: -4.0, rssi: -107.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 3932
time: 2019-05-23 13:16:09.262000, freq: 868.1, chan: 21, lsnr: 0.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4174
time: 2019-05-23 13:25:18.812000, freq: 868.5, chan: 23, lsnr: -3.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3931
time: 2019-05-23 13:25:31.140000, freq: 868.3, chan: 22, lsnr: -8.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3931
time: 2019-05-23 13:37:47.679000, freq: 868.3, chan: 22, lsnr: 1.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 24, FCnt: 4002
time: 2019-05-23 13:38:15.481000, freq: 868.3, chan: 22, lsnr: 1.0, rssi: -111.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 4003
time: 2019-05-23 13:39:20.325000, freq: 868.3, chan: 22, lsnr: -3.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4182
time: 2019-05-23 13:46:05.040000, freq: 868.1, chan: 21, lsnr: 13.0, rssi: -84.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 23, FCnt: 4133
time: 2019-05-23 13:47:52.866000, freq: 868.1, chan: 21, lsnr: 15.0, rssi: -89.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 231


QUESTION: How many packets are received for each device?

ANSWER:
For DEVICE dev_eui:120 all received frames are 2.
For DEVICE dev_eui:121 all received frames are 2.
For DEVICE dev_eui:122 all received frames are 1.
For DEVICE dev_eui:123 all received frames are 3.
For DEVICE dev_eui:124 all received frames are 4.
For DEVICE dev_eui:126 all received frames are 2.
For DEVICE dev_eui:127 all received frames are 2.
For DEVICE dev_eui:128 all received frames are 1.

"""

question, answer = preprocess_sample(file_contents)

print(question['content'])
print(answer['content'])

Given the following LOG, how many frames are received per each dev_eui?

LOG:

time: 2019-05-23 12:16:10.162000, freq: 868.1, chan: 21, lsnr: 8.0, rssi: -105.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4173
time: 2019-05-23 12:18:43.187000, freq: 868.5, chan: 23, lsnr: -6.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4186
time: 2019-05-23 12:25:19.676000, freq: 868.5, chan: 23, lsnr: -2.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3930
time: 2019-05-23 12:25:32.075000, freq: 868.3, chan: 22, lsnr: -7.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3930
time: 2019-05-23 12:37:55.014000, freq: 868.5, chan: 23, lsnr: 4.0, rssi: -110.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 4001
time: 2019-05-23 12:39:21.253000, freq: 868.1, chan: 21, lsnr: 0.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4181
time: 2019-05-23 12:46:05.929000, freq: 86

# Create the few-shots samples.

In [10]:
def create_few_shots_set(folder_path,nr_samples):

    # Counter to keep track of the number of files read
    count = 0

    few_shots_set = []

    # Get a list of all files in the directory
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

    # Shuffle the list of files. Seed set - equal to 4 - for reproducibility.
    random.Random(4).shuffle(files)

    global test_set
    test_set = files

    # Iterate through all files in the folder
    for filename in files:
        print(filename)
        # Check if the file is a text file
        if filename.endswith(".txt"):
            # Open the file and read its contents
            with open(os.path.join(folder_path, filename), 'r') as file:
                # Read the contents of the file and append it to all_contents
                file_contents = file.read()

                question, answer = preprocess_sample(file_contents)

                few_shots_set.append(question)
                few_shots_set.append(answer)


                # Increment the counter
                count += 1

                # Check if reached the limit of n files
                if count == n:
                    break
        test_set.remove(filename)

    return few_shots_set

In [11]:
# Specify the number of files to read
n = 3  # Example: Read the first 3 files

# Call the function to read text files in the folder and concatenate their contents
few_shots_samples = create_few_shots_set(folder_path, n)

# Print or use the concatenated contents as needed
print(few_shots_samples)

prompt_n_hours=2_skip_hours=42.txt
prompt_n_hours=2_skip_hours=480.txt
prompt_n_hours=2_skip_hours=1016.txt
[{'role': 'user', 'content': 'Given the following LOG, how many frames are received per each dev_eui?\n\nLOG:\ntime: 2019-05-21 18:16:47.648000, freq: 868.5, chan: 23, lsnr: 5.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4130\ntime: 2019-05-21 18:19:00.647000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4142\ntime: 2019-05-21 18:25:56.343000, freq: 868.1, chan: 21, lsnr: -2.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886\ntime: 2019-05-21 18:26:02.544000, freq: 868.1, chan: 21, lsnr: -7.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886\ntime: 2019-05-21 18:26:08.678000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886\ntime: 2019-05-21 18:38:09.529000, freq: 86

# Chat with the model to get the results.

In [14]:
# Define the directory path containing the results.
folder_name = "log_knowledge_extraction/codegemma_7b_it_Q1_results/"
directory_path = os.path.join(os.getcwd(), folder_name)

# Iterate through all the test_files
for filename in test_set:
        print(filename)
        if(os.path.isfile(directory_path+"/"+filename)):
          print("Already done!\n\n")
          continue
        # Check if the file is a text file
        if filename.endswith(".txt"):

            # Create the directory if it doesn't exist
            if not os.path.exists(directory_path):
                os.makedirs(directory_path)

            # Open the file and read its contents
            with open(os.path.join(folder_path, filename), 'r') as file:
                # Read the contents of the file and append it to all_contents
                file_contents = file.read()
                chat = ""
                # Specify the number of files to read
                n = 3  # Example: Read the first 5 files
                # Call the function to read text files in the folder and concatenate their contents
                few_shots_samples = create_few_shots_set(folder_path, n)

                chat = few_shots_samples
                question,anwer = preprocess_sample(file_contents)

                chat.append(question)

                prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
                print("****.  ECCCO IL PROMPT !!!! **** ")
                print(prompt)
                print("\n\n\n\n")

                inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
                outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=1000)
                result = tokenizer.decode(outputs[0])
                print("****.  ECCCO LA RISPOSTA !!!! **** ")
                print(result)

                # Create the directory if it doesn't exist
                if not os.path.exists(directory_path):
                    os.makedirs(directory_path)

                # Define the file path
                file_path = os.path.join(directory_path, filename)

                with open(file_path, 'w') as f:
                  # Write some content to the file
                  f.write(result)
        break

prompt_n_hours=2_skip_hours=36.txt
prompt_n_hours=2_skip_hours=42.txt
prompt_n_hours=2_skip_hours=480.txt
prompt_n_hours=2_skip_hours=1016.txt
****.  ECCCO IL PROMPT !!!! **** 
<bos><start_of_turn>user
Given the following LOG, how many frames are received per each dev_eui?

LOG:
time: 2019-05-21 18:16:47.648000, freq: 868.5, chan: 23, lsnr: 5.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4130
time: 2019-05-21 18:19:00.647000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4142
time: 2019-05-21 18:25:56.343000, freq: 868.1, chan: 21, lsnr: -2.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:26:02.544000, freq: 868.1, chan: 21, lsnr: -7.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:26:08.678000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 2

In [ ]:
!zip -r mistral_7b_Q3_results.zip mistral_7b_Q3_results/

  adding: mistral_7b_Q3_results/ (stored 0%)
  adding: mistral_7b_Q3_results/prompt_n_hours=2_skip_hours=478.txt (deflated 80%)
  adding: mistral_7b_Q3_results/prompt_n_hours=2_skip_hours=718.txt (deflated 80%)
  adding: mistral_7b_Q3_results/prompt_n_hours=2_skip_hours=310.txt (deflated 78%)
  adding: mistral_7b_Q3_results/prompt_n_hours=2_skip_hours=634.txt (deflated 78%)
  adding: mistral_7b_Q3_results/prompt_n_hours=2_skip_hours=464.txt (deflated 80%)
  adding: mistral_7b_Q3_results/prompt_n_hours=2_skip_hours=760.txt (deflated 65%)
  adding: mistral_7b_Q3_results/prompt_n_hours=2_skip_hours=324.txt (deflated 78%)
  adding: mistral_7b_Q3_results/prompt_n_hours=2_skip_hours=828.txt (deflated 80%)
  adding: mistral_7b_Q3_results/prompt_n_hours=2_skip_hours=144.txt (deflated 78%)
  adding: mistral_7b_Q3_results/prompt_n_hours=2_skip_hours=582.txt (deflated 79%)
  adding: mistral_7b_Q3_results/prompt_n_hours=2_skip_hours=304.txt (deflated 68%)
  adding: mistral_7b_Q3_results/prompt_n_h

# Given the same few-shot samples ask to Mistral to generate the Python code to perform the task.

In [15]:
prompt ="""
Given the following LOG string, write a Python function to calculate and output (print and store in a file) how many frames are received per each dev_eui.

LOG:
time: 2019-05-21 18:16:47.648000, freq: 868.5, chan: 23, lsnr: 5.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4130
time: 2019-05-21 18:19:00.647000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4142
time: 2019-05-21 18:25:56.343000, freq: 868.1, chan: 21, lsnr: -2.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:26:02.544000, freq: 868.1, chan: 21, lsnr: -7.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:26:08.678000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:38:09.529000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 3958
time: 2019-05-21 18:39:57.539000, freq: 868.3, chan: 22, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4137
time: 2019-05-21 18:40:03.735000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4137
time: 2019-05-21 18:46:42.609000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -92.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 25, FCnt: 4088
time: 2019-05-21 18:48:29.241000, freq: 868.1, chan: 21, lsnr: 14.0, rssi: -90.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 186
time: 2019-05-21 19:16:46.731000, freq: 868.3, chan: 22, lsnr: 5.0, rssi: -108.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4131
time: 2019-05-21 19:19:00.174000, freq: 868.5, chan: 23, lsnr: -8.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4143
time: 2019-05-21 19:25:55.447000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:26:01.582000, freq: 868.5, chan: 23, lsnr: -5.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:26:07.785000, freq: 868.5, chan: 23, lsnr: -7.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:38:09.158000, freq: 868.5, chan: 23, lsnr: -1.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 25, FCnt: 3959
time: 2019-05-21 19:39:56.692000, freq: 868.1, chan: 21, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4138
time: 2019-05-21 19:40:02.892000, freq: 868.1, chan: 21, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4138
time: 2019-05-21 19:46:41.701000, freq: 868.5, chan: 23, lsnr: 8.0, rssi: -106.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 25, FCnt: 4089
time: 2019-05-21 19:48:28.310000, freq: 868.1, chan: 21, lsnr: 13.0, rssi: -87.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 187
time: 2019-05-21 20:12:05.544000, freq: 868.5, chan: 23, lsnr: 0.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 3889


OUTPUT:
For DEVICE dev_eui:120 all received frames are 2.
For DEVICE dev_eui:121 all received frames are 4.
For DEVICE dev_eui:122 all received frames are 2.
For DEVICE dev_eui:123 all received frames are 2.
For DEVICE dev_eui:124 all received frames are 6.
For DEVICE dev_eui:126 all received frames are 2.
For DEVICE dev_eui:127 all received frames are 2.
For DEVICE dev_eui:128 all received frames are 1.

"""

In [16]:
chat_new = [
    { "role": "user", "content": prompt }
]


prompt = tokenizer.apply_chat_template(chat_new, tokenize=False, add_generation_prompt=True)

print(prompt)

inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=5000)
print(tokenizer.decode(outputs[0]))

<bos><start_of_turn>user
Given the following LOG string, write a Python function to calculate and output (print and store in a file) how many frames are received per each dev_eui.

LOG:
time: 2019-05-21 18:16:47.648000, freq: 868.5, chan: 23, lsnr: 5.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4130
time: 2019-05-21 18:19:00.647000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4142
time: 2019-05-21 18:25:56.343000, freq: 868.1, chan: 21, lsnr: -2.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:26:02.544000, freq: 868.1, chan: 21, lsnr: -7.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:26:08.678000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:38:09.529000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -113.0

In [18]:
import re

LOG = """
time: 2019-05-21 18:16:47.648000, freq: 868.5, chan: 23, lsnr: 5.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4130
time: 2019-05-21 18:19:00.647000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4142
time: 2019-05-21 18:25:56.343000, freq: 868.1, chan: 21, lsnr: -2.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:26:02.544000, freq: 868.1, chan: 21, lsnr: -7.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:26:08.678000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:38:09.529000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 3958
time: 2019-05-21 18:39:57.539000, freq: 868.3, chan: 22, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4137
time: 2019-05-21 18:40:03.735000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4137
time: 2019-05-21 18:46:42.609000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -92.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 25, FCnt: 4088
time: 2019-05-21 18:48:29.241000, freq: 868.1, chan: 21, lsnr: 14.0, rssi: -90.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 186
time: 2019-05-21 19:16:46.731000, freq: 868.3, chan: 22, lsnr: 5.0, rssi: -108.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4131
time: 2019-05-21 19:19:00.174000, freq: 868.5, chan: 23, lsnr: -8.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4143
time: 2019-05-21 19:25:55.447000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:26:01.582000, freq: 868.5, chan: 23, lsnr: -5.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:26:07.785000, freq: 868.5, chan: 23, lsnr: -7.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:38:09.158000, freq: 868.5, chan: 23, lsnr: -1.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 25, FCnt: 3959
time: 2019-05-21 19:39:56.692000, freq: 868.1, chan: 21, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4138
time: 2019-05-21 19:40:02.892000, freq: 868.1, chan: 21, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4138
time: 2019-05-21 19:46:41.701000, freq: 868.5, chan: 23, lsnr: 8.0, rssi: -106.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 25, FCnt: 4089
time: 2019-05-21 19:48:28.310000, freq: 868.1, chan: 21, lsnr: 13.0, rssi: -87.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 187
time: 2019-05-21 20:12:05.544000, freq: 868.5, chan: 23, lsnr: 0.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 3889
"""

def process_log(log):
  result=""
  # Create a dictionary to store the number of frames received for each device
  device_frames = {}

  # Extract the device eui and number of frames from the log string
  for line in LOG.splitlines():
      match = re.search(r"dev_eui: (\d+),", line)
      if match:
          dev_eui = int(match.group(1))
          if dev_eui not in device_frames:
              device_frames[dev_eui] = 0
          device_frames[dev_eui] += 1

  # Print the number of frames received for each device
  for dev_eui, frames in device_frames.items():
      print(f"For DEVICE dev_eui:{dev_eui} all received frames are {frames}.")

  ## Store the output in a file
  #with open("device_frames.txt", "w") as f:
  for dev_eui, frames in device_frames.items():
        result+=(f"For DEVICE dev_eui:{dev_eui} all received frames are {frames}.\n")
          #f.write(f"For DEVICE dev_eui:{dev_eui} all received frames are {frames}.\n")

  return result


## Let's run the code.

In [19]:
# Define the directory path containing the results.
folder_name = 'log_knowledge_extraction/codegemma-7b-it_source_code_results/'
#directory_path = os.path.join(os.getcwd(), folder_name)
# Create the directory if it doesn't exist
if not os.path.exists(folder_name):
  os.makedirs(folder_name)

# Iterate through all the test_files
for filename in test_set:
        file_path = folder_name+filename
        print(filename)
        print(file_path)
        if(os.path.isfile(file_path)):
          print("Already done!\n\n")
          continue
        # Check if the file is a text file
        if filename.endswith(".txt"):

            # Open the file and read its contents
            with open(os.path.join(folder_path, filename), 'r') as file:
                # Read the contents of the file and append it to all_contents
                file_contents = file.read()

                # Remove the ANSWER in the original log file.
                file_contents = file_contents.split("ANSWER", 1)

                result = process_log(file_contents[0])

                print(result+"\n\n")

                with open(file_path, 'w') as f:
                  # Write some content to the file
                  f.write(result)

Streaming output truncated to the last 5000 lines.


prompt_n_hours=2_skip_hours=232.txt
log_knowledge_extraction/codegemma-7b-it_source_code_results/prompt_n_hours=2_skip_hours=232.txt
For DEVICE dev_eui:126 all received frames are 2.
For DEVICE dev_eui:122 all received frames are 2.
For DEVICE dev_eui:124 all received frames are 6.
For DEVICE dev_eui:123 all received frames are 2.
For DEVICE dev_eui:121 all received frames are 4.
For DEVICE dev_eui:127 all received frames are 2.
For DEVICE dev_eui:120 all received frames are 2.
For DEVICE dev_eui:128 all received frames are 1.
For DEVICE dev_eui:126 all received frames are 2.
For DEVICE dev_eui:122 all received frames are 2.
For DEVICE dev_eui:124 all received frames are 6.
For DEVICE dev_eui:123 all received frames are 2.
For DEVICE dev_eui:121 all received frames are 4.
For DEVICE dev_eui:127 all received frames are 2.
For DEVICE dev_eui:120 all received frames are 2.
For DEVICE dev_eui:128 all received frames are 1.



prompt_n_ho

# Here the function to run the evaluation at file-level and at global-level.

In [20]:
def evaluate_results (dataset_path,result_path):
  dataset_corpus = ""
  results_corpus = ""
  # Iterate through all the test_files
  for filename in test_set:
      # Specify the paths to the files
      file1_path = result_path+filename
      file2_path = dataset_path+filename

      # Read the contents of the files
      try:
        with open(file1_path, 'r', encoding='utf-8') as file1:
          text1 = file1.read()
      except:
        continue
      try:
        with open(file2_path, 'r', encoding='utf-8') as file2:
          text2 = file2.read()
      except:
        continue

      # Find the index where "ANSWER:" starts
      answer_start_index = text2.find("ANSWER:")

      # Remove all content before and excluding "ANSWER:"
      if answer_start_index != -1:
          text2 = text2[answer_start_index + len("ANSWER:"):].strip()

      print("Sample: ", filename)

      print(text2)
      print("\n")
      print(text1)

      dataset_corpus += text2
      results_corpus += text1

      # Tokenize the texts
      reference = text1.split()
      candidate = text2.split()

      # Compute BLEU score
      smoothing_function = SmoothingFunction().method1
      bleu_score = sentence_bleu([reference], candidate, smoothing_function=smoothing_function)

      # Compute ROUGE score
      scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
      rouge_scores = scorer.score(text1, text2)

      # Print the results
      print(f"BLEU score: {bleu_score}")
      print(f"ROUGE-1: {rouge_scores['rouge1']}")
      print(f"ROUGE-L: {rouge_scores['rougeL']}")

      print("\n\n")
  # Now let's print the results on the whole corpus.
  print("\n\nTHE RESULT ON THE TWO CORPORA ARE:\n")
  # Tokenize the texts
  reference = results_corpus.split()
  candidate = dataset_corpus.split()

  # Compute BLEU score
  smoothing_function = SmoothingFunction().method1
  bleu_score = sentence_bleu([reference], candidate, smoothing_function=smoothing_function)

  # Compute ROUGE score
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
  rouge_scores = scorer.score(text1, text2)

  # Print the results
  print(f"BLEU score: {bleu_score}")
  print(f"ROUGE-1: {rouge_scores['rouge1']}")
  print(f"ROUGE-L: {rouge_scores['rougeL']}")

## Let's evaluate the results of CodeLlama

In [ ]:
dataset_path = "prompt_Q2/"
result_path = "mistral_7b_Q2_results/"
evaluate_results(dataset_path,result_path)

Streaming output truncated to the last 5000 lines.
Sample:  prompt_n_hours=2_skip_hours=318.txt
dev_eui: 124 
 dev_eui: 128


dev_eui: 122, dev_eui: 124, dev_eui: 128
BLEU score: 0.11404605374835301
ROUGE-1: Score(precision=1.0, recall=0.6666666666666666, fmeasure=0.8)
ROUGE-L: Score(precision=1.0, recall=0.6666666666666666, fmeasure=0.8)



Sample:  prompt_n_hours=2_skip_hours=390.txt
dev_eui: 124 
 dev_eui: 126 
 dev_eui: 128


dev_eui: 123, dev_eui: 128, dev_eui: 124, dev_eui: 126
BLEU score: 0.07356556000975763
ROUGE-1: Score(precision=1.0, recall=0.75, fmeasure=0.8571428571428571)
ROUGE-L: Score(precision=0.6666666666666666, recall=0.5, fmeasure=0.5714285714285715)



Sample:  prompt_n_hours=2_skip_hours=710.txt
dev_eui: 120 
 dev_eui: 128


dev_eui: 122, dev_eui: 124, dev_eui: 128
BLEU score: 0.11404605374835301
ROUGE-1: Score(precision=0.8333333333333334, recall=0.5555555555555556, fmeasure=0.6666666666666667)
ROUGE-L: Score(precision=0.8333333333333334, recall=0.555555555555555

## Let's evaluate the results of the source code generated by CodeLlama.

In [22]:
dataset_path = "log_knowledge_extraction/prompt_Q1-2024-05-20/"
result_path = "log_knowledge_extraction/codegemma-7b-it_source_code_results/"
evaluate_results(dataset_path,result_path)

Streaming output truncated to the last 5000 lines.

Sample:  prompt_n_hours=2_skip_hours=32.txt
For DEVICE dev_eui:120 all received frames are 2.
For DEVICE dev_eui:121 all received frames are 3.
For DEVICE dev_eui:122 all received frames are 1.
For DEVICE dev_eui:123 all received frames are 2.
For DEVICE dev_eui:124 all received frames are 1.
For DEVICE dev_eui:126 all received frames are 1.
For DEVICE dev_eui:127 all received frames are 3.
For DEVICE dev_eui:128 all received frames are 3.


For DEVICE dev_eui:126 all received frames are 2.
For DEVICE dev_eui:122 all received frames are 2.
For DEVICE dev_eui:124 all received frames are 6.
For DEVICE dev_eui:123 all received frames are 2.
For DEVICE dev_eui:121 all received frames are 4.
For DEVICE dev_eui:127 all received frames are 2.
For DEVICE dev_eui:120 all received frames are 2.
For DEVICE dev_eui:128 all received frames are 1.

BLEU score: 0.7928515635589939
ROUGE-1: Score(precision=0.9375, recall=0.9375, fmeasure=0.9375)
ROUGE